In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
from __future__ import print_function
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Layer
from tensorflow.keras import activations
from tensorflow.keras import utils
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import model_from_json
from sklearn.model_selection import KFold, StratifiedKFold
import json
import gc
import tensorflow as tf

import cv2
from sklearn.utils import shuffle
import random


In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
# Determine constants

labels = ['live', 'spoof']
data_prabhat = r'D:\LivenessDetectionDatasets\antispoofingDataPrabhat'
data_nuaa_detected = r'D:\LivenessDetectionDatasets\NUAA-Detectedface'
data_nuaa_raw = r'D:\LivenessDetectionDatasets\NUAA-raw'
image_width=image_height = 224
n_color_sector = 3
colors_dark = ["#1F1F1F", "#313131", '#636363', '#AEAEAE', '#DADADA']
colors_red = ["#331313", "#582626", '#9E1717', '#D35151', '#E9B4B4']
colors_green = ['#01411C','#4B6F44','#4F7942','#74C365','#D0F0C0']

In [4]:
def data_converter_from_path(data_paths):
    X= []
    y = []
    
    for data_path in data_paths:
        for label in labels:
            join_label = os.path.join(data_path, label)
            for image_folder in os.listdir(join_label):
                if image_folder[:2]=='00':
                    join_image_folder = os.path.join(join_label, image_folder)
                for image in os.listdir(join_image_folder):
                    join_image = os.path.join(join_image_folder, image)
                    if (join_image[-3:]=='jpg')|(join_image[-3:]=='png'):
                        img = cv2.imread(join_image)
                        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                        img = cv2.resize(img, (image_width, image_height))
                        X.append(img)
                        y.append(labels.index(label))
    return X, y

data_paths = [data_nuaa_detected]
X, y = data_converter_from_path(data_paths)  
    
def data_converter_from_simple(data_paths):
    for data_path in data_paths:
        for label in labels:
            join_label = os.path.join(data_path, label)
            for image in os.listdir(join_label):
                join_image = os.path.join(join_label, image)
                if (join_image[-3:]=='jpg')|(join_image[-3:]=='png'):
                    img = cv2.imread(join_image)
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    img = cv2.resize(img, (image_width, image_height))
                    X.append(img)
                    y.append(labels.index(label))
    return X, y
    

In [5]:
X = np.array(X)
y = np.array(y)
X, y = shuffle(X,y, random_state=123)

In [8]:
print(np.array(X).shape, np.array(y).shape)

((12732, 224, 224, 3), (12732,))

In [6]:
from sklearn.model_selection import train_test_split

print(X.shape)
print(y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

print(f'Training dataset size of X_train: {len(X_train)}')
print(f'Testing dataset size of X_test: {len(X_test)}')
print(f'Testing dataset size of y_train: {len(y_train)}')
print(f'Testing dataset size of y_test: {len(y_test)}')
print(f'Testing dataset size of X_test: {len(X_val)}')
print(f'Testing dataset size of y_train: {len(y_val)}')

(12732, 224, 224, 3)
(12732,)
Training dataset size of X_train: 7129
Testing dataset size of X_test: 3820
Testing dataset size of y_train: 7129
Testing dataset size of y_test: 3820
Testing dataset size of X_test: 1783
Testing dataset size of y_train: 1783


In [14]:
def visualize_images(X,y, amount_of_images, rows, cols):
    list_of_images = random.sample(range(0, len(y)), amount_of_images)
    fig  = plt.figure(figsize =(30,30))
    for i in range(1, rows*cols+1):
        fig.add_subplot(rows, cols, i)
        img_array = X[list_of_images[i-1]]
        plt.imshow(img_array)
        plt.xlabel(labels[y[list_of_images[i-1]]])
    plt.show()
    

In [ ]:
visualize_images(X,y, 25, 5, 5)

In [7]:
# the squashing function.
# we use 0.5 in stead of 1 in hinton's paper.
# if 1, the norm of vector will be zoomed out.
# if 0.5, the norm will be zoomed in while original norm is less than 0.5
# and be zoomed out while original norm is greater than 0.5.
def squash(x, axis=-1):
    s_squared_norm = K.sum(K.square(x), axis, keepdims=True) + K.epsilon()
    scale = K.sqrt(s_squared_norm) / (0.5 + s_squared_norm)
    return scale * x


# define our own softmax function instead of K.softmax
# because K.softmax can not specify axis.
def softmax(x, axis=-1):
    ex = K.exp(x - K.max(x, axis=axis, keepdims=True))
    return ex / K.sum(ex, axis=axis, keepdims=True)


# define the margin loss like hinge loss
def margin_loss(y_true, y_pred):
    lamb, margin = 0.5, 0.1
    return K.sum(y_true * K.square(K.relu(1 - margin - y_pred)) + lamb * (
        1 - y_true) * K.square(K.relu(y_pred - margin)), axis=-1)


class Capsule(Layer):
    """A Capsule Implement with Pure Keras
    There are two vesions of Capsule.
    One is like dense layer (for the fixed-shape input),
    and the other is like timedistributed dense (for various length input).

    The input shape of Capsule must be (batch_size,
                                        input_num_capsule,
                                        input_dim_capsule
                                       )
    and the output shape is (batch_size,
                             num_capsule,
                             dim_capsule
                            )

    Capsule Implement is from https://github.com/bojone/Capsule/
    Capsule Paper: https://arxiv.org/abs/1710.09829
    """

    def __init__(self,
                 num_capsule,
                 dim_capsule,
                 routings=3,
                 share_weights=True,
                 activation='squash',
                 **kwargs):
        super(Capsule, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.share_weights = share_weights
        if activation == 'squash':
            self.activation = squash
        else:
            self.activation = activations.get(activation)

    def build(self, input_shape):
        input_dim_capsule = input_shape[-1]
        if self.share_weights:
            self.kernel = self.add_weight(
                name='capsule_kernel',
                shape=(1, input_dim_capsule,
                       self.num_capsule * self.dim_capsule),
                initializer='glorot_uniform',
                trainable=True)
        else:
            input_num_capsule = input_shape[-2]
            self.kernel = self.add_weight(
                name='capsule_kernel',
                shape=(input_num_capsule, input_dim_capsule,
                       self.num_capsule * self.dim_capsule),
                initializer='glorot_uniform',
                trainable=True)

    def call(self, inputs):
        """Following the routing algorithm from Hinton's paper,
        but replace b = b + <u,v> with b = <u,v>.

        This change can improve the feature representation of Capsule.

        However, you can replace
            b = K.batch_dot(outputs, hat_inputs, [2, 3])
        with
            b += K.batch_dot(outputs, hat_inputs, [2, 3])
        to realize a standard routing.
        """

        if self.share_weights:
            hat_inputs = K.conv1d(inputs, self.kernel)
        else:
            hat_inputs = K.local_conv1d(inputs, self.kernel, [1], [1])

        batch_size = K.shape(inputs)[0]
        input_num_capsule = K.shape(inputs)[1]
        hat_inputs = K.reshape(hat_inputs,
                               (batch_size, input_num_capsule,
                                self.num_capsule, self.dim_capsule))
        hat_inputs = K.permute_dimensions(hat_inputs, (0, 2, 1, 3))

        b = K.zeros_like(hat_inputs[:, :, :, 0])
        for i in range(self.routings):
            c = softmax(b, 1)
            o = self.activation(K.batch_dot(c, hat_inputs, [2, 2]))
            if i < self.routings - 1:
                b = K.batch_dot(o, hat_inputs, [2, 3])
                if K.backend() == 'theano':
                    o = K.sum(o, axis=1)

        return o

    def compute_output_shape(self, input_shape):
        return (None, self.num_capsule, self.dim_capsule)

In [8]:
x_train = X_train.astype(float)
x_test = X_test.astype(float)
x_val = X_val.astype(float)

In [9]:
batch_size = 16
epochs = 100
dshape = 1
routing = 3

# x_train = X_train.astype('float32')
# x_test = X_test.astype('float32')
x_train /= 255
x_test /= 255
x_val /= 255
Y_train = utils.to_categorical(y_train.flatten(), 2)
Y_test = utils.to_categorical(y_test.flatten(), 2)
Y_val = utils.to_categorical(y_val.flatten(), 2)

In [10]:
def cap_batch_dot(x,y,axis):
    xax=axis[0]-1
    xdim=x.shape[axis[0]-1]
    
    x1=tf.split(x,num_or_size_splits=xdim ,axis=xax)
    y1=tf.split(y,num_or_size_splits=xdim ,axis=xax)
    
    xy_batch=[]
    for i in range(xdim):
        #print("x1=",x1[i])
        #print("y1=",y1[i])
        xy_dot = K.batch_dot(x1[i], y1[i], axes=axis)
        #print(xy_dot)
        xy_batch.append(xy_dot)
    xy_batch_dot=K.concatenate(xy_batch, axis=xax)
    xy_batch_dot=K.squeeze(xy_batch_dot, axis=axis[0])
    return xy_batch_dot

In [11]:
def squash(x, axis=-1):
    s_squared_norm = K.sum(K.square(x), axis, keepdims=True) + K.epsilon()
    scale = K.sqrt(s_squared_norm) / (0.5 + s_squared_norm)
    return scale * x


def softmax(x, axis=-1):
    ex = K.exp(x - K.max(x, axis=axis, keepdims=True))
    return ex / K.sum(ex, axis=axis, keepdims=True)


def margin_loss(y_true, y_pred):
    lamb, margin = 0.5, 0.1
    return K.sum(y_true * K.square(K.relu(1 - margin - y_pred)) + lamb * (1 - y_true) * K.square(K.relu(y_pred - margin)), axis=-1)

In [12]:
class Capsule(Layer):

    def __init__(self,num_capsule,dim_capsule,routings=3,share_weights=True,activation='squash',**kwargs):
        
        super(Capsule, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.share_weights = share_weights
        self.activation = squash
        
        
    def build(self, input_shape):
        
        input_dim_capsule = input_shape[-1]
        self.kernel = self.add_weight(name='capsule_kernel',
                                      shape=(1, input_dim_capsule,self.num_capsule * self.dim_capsule),
                                      initializer='glorot_uniform',
                                      trainable=True)

    def call(self, inputs, training=None):
        
        hat_inputs = K.conv1d(inputs, self.kernel)     
        batch_size = K.shape(inputs)[0]
        input_num_capsule = K.shape(inputs)[1]
        
        hat_inputs = K.reshape(hat_inputs,(batch_size, input_num_capsule, self.num_capsule, self.dim_capsule))
        hat_inputs = K.permute_dimensions(hat_inputs, (0, 2, 1, 3))
        #print("hat_inputs=",hat_inputs)
        
        b = K.zeros_like(hat_inputs[:, :, :, 0])
        #print("b=",b)

        for i in range(self.routings):
            c = softmax(b,axis=1)
            #print(c)
            o_batchdot = cap_batch_dot(c, hat_inputs, (2, 2))
            #print("outputs_batch=",o_batchdot)
            o = self.activation(o_batchdot)
            #print("o=",o)
            
            if i < self.routings - 1:
                b = cap_batch_dot(o, hat_inputs, [2, 3])
                #print("b=",b)
        return o

    def compute_output_shape(self, input_shape):
        return (None, self.num_capsule, self.dim_capsule)
    
    def get_config(self):
        config = super().get_config()
        config['num_capsule'] = self.num_capsule
        config['dim_capsule'] = self.dim_capsule
        return config

In [13]:
batch_size = 16
num_classes = 2
epochs = 70

In [14]:
input_image = Input(shape=(224, 224, 3))
x = Conv2D(64, (3, 3), activation='relu')(input_image)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = AveragePooling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu')(x)
x = Conv2D(128, (3, 3), activation='relu')(x)
x = Reshape((-1, 128))(x)
capsule = Capsule(2, 16, 3, True)(x)
print("capsule=",capsule)

capsule= KerasTensor(type_spec=TensorSpec(shape=(None, 2, 16), dtype=tf.float32, name=None), name='capsule/mul_2:0', description="created by layer 'capsule'")


In [15]:
output = Lambda(lambda z: K.sqrt(K.sum(K.square(z), 2)))(capsule)
print("output=",output)
model = Model(inputs=input_image, outputs=output)

output= KerasTensor(type_spec=TensorSpec(shape=(None, 2), dtype=tf.float32, name=None), name='lambda/Sqrt:0', description="created by layer 'lambda'")


In [16]:
model.compile(loss=margin_loss, optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 222, 222, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 220, 220, 64)      36928     
_________________________________________________________________
average_pooling2d (AveragePo (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 108, 108, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 106, 106, 128)     147584    
_________________________________________________________________
reshape (Reshape)            (None, 11236, 128)        0     

In [17]:
datagen = ImageDataGenerator(
        rotation_range=180,  
        horizontal_flip=True,  
        vertical_flip=True)
datagen.fit(x_train)                               

In [18]:
batch_size = 16
num_classes = 2
epochs = 100


In [20]:
history = model.fit(
        x_train,
        Y_train,
        batch_size=batch_size,
        validation_data=(x_val, Y_val),
        epochs=epochs,
        shuffle=True)

Epoch 1/100
446/446 [==============================] - 231s 470ms/step - loss: 0.4050 - accuracy: 0.4542 - val_loss: 0.4050 - val_accuracy: 0.4436
Epoch 2/100
446/446 [==============================] - 131s 295ms/step - loss: 0.4050 - accuracy: 0.4545 - val_loss: 0.4050 - val_accuracy: 0.4492
Epoch 3/100
446/446 [==============================] - 131s 293ms/step - loss: 0.4050 - accuracy: 0.4650 - val_loss: 0.4050 - val_accuracy: 0.4520
Epoch 4/100
446/446 [==============================] - 131s 294ms/step - loss: 0.4050 - accuracy: 0.4564 - val_loss: 0.4050 - val_accuracy: 0.4341
Epoch 5/100
446/446 [==============================] - 130s 292ms/step - loss: 0.4050 - accuracy: 0.4593 - val_loss: 0.4050 - val_accuracy: 0.4492
Epoch 6/100
446/446 [==============================] - 155s 348ms/step - loss: 0.4050 - accuracy: 0.4615 - val_loss: 0.4050 - val_accuracy: 0.4453
Epoch 7/100
446/446 [==============================] - 169s 376ms/step - loss: 0.4050 - accuracy: 0.4521 - val_loss: 0

KeyboardInterrupt: 

In [23]:
x_test.shape

(3820, 224, 224, 3)

In [26]:
pred = model.predict(x_test)

array([[[[0.16862745, 0.16470588, 0.15686275],
         [0.3254902 , 0.32156863, 0.31372549],
         [0.45882353, 0.45490196, 0.44705882],
         ...,
         [0.10980392, 0.10588235, 0.12941176],
         [0.10980392, 0.10588235, 0.12941176],
         [0.10980392, 0.10588235, 0.1254902 ]],

        [[0.25882353, 0.25490196, 0.24705882],
         [0.37647059, 0.37254902, 0.36470588],
         [0.46666667, 0.4627451 , 0.45490196],
         ...,
         [0.10980392, 0.10588235, 0.13333333],
         [0.10980392, 0.10588235, 0.12941176],
         [0.10980392, 0.10588235, 0.12941176]],

        [[0.43921569, 0.43529412, 0.42745098],
         [0.49019608, 0.48627451, 0.47843137],
         [0.50588235, 0.50196078, 0.49411765],
         ...,
         [0.11372549, 0.10980392, 0.13333333],
         [0.11372549, 0.10980392, 0.12941176],
         [0.11372549, 0.10980392, 0.1254902 ]],

        ...,

        [[0.15294118, 0.14901961, 0.18431373],
         [0.14901961, 0.14509804, 0.18039216]

In [25]:

loss_test, acc_test = model.evaluate(x_test, y_test)

print ("\n\n================================\n\n")
print ("Loss on Test Set: {}".format(loss_test))
print ("Accuracy on Test Set: {0:.2f} %".format(acc_test * 100))
print ("\n\n================================\n\n")

KeyboardInterrupt: 

In [83]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
reduce_lr = ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.3, patience = 4, min_delta = 0.001,
                              mode='auto',verbose=1)

In [84]:
epochs=35
k = 3
kfold = KFold(n_splits=k, shuffle=True, random_state=42)
cvscores = []
historieskv = []
Fold = 1
for train, val in kfold.split(np.array(X_train), y_train):
    gc.collect()
    K.clear_session()
    
    x_train = np.array(X_train)[train]
    x_val = np.array(X_train)[val]
    
    Y_train = y_train[train]
    Y_val = y_train[val]
     
    print("=========================================")
    print("====== K Fold Validation step => %d/%d =======" % (Fold,k))
    print("=========================================")

    historykv=model.fit(x_train, Y_train, 
              validation_data=(x_val, Y_val),
              epochs=epochs,
              batch_size=16,
              callbacks=[es, reduce_lr], verbose=1)
    historieskv.append(historykv)
    
    
    # evaluate the model
    scores = model.evaluate(x_val, Y_val, verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    
    
    Fold = Fold +1

print("%s: %.2f%%" % ("Mean Accuracy of CV scores: ",np.mean(cvscores)))
print("%s: %.2f%%" % ("Standard Deviation of CV scores: +/-", np.std(cvscores)))

loss_test, acc_test = model.evaluate(X_test, y_test)

print ("\n\n================================\n\n")
print ("Loss on Test Set: {}".format(loss_test))
print ("Accuracy on Test Set: {0:.2f} %".format(acc_test * 100))
print ("\n\n================================\n\n")

====== K Fold Validation step => 1/3 =======
Epoch 1/35
297/297 [==============================] - 90s 301ms/step - loss: 0.0000e+00 - accuracy: 0.4179 - val_loss: 0.0000e+00 - val_accuracy: 0.4161
Epoch 2/35
297/297 [==============================] - 89s 298ms/step - loss: 0.0000e+00 - accuracy: 0.4179 - val_loss: 0.0000e+00 - val_accuracy: 0.4161
Epoch 3/35
297/297 [==============================] - 89s 298ms/step - loss: 0.0000e+00 - accuracy: 0.4179 - val_loss: 0.0000e+00 - val_accuracy: 0.4161
Epoch 4/35
297/297 [==============================] - 89s 298ms/step - loss: 0.0000e+00 - accuracy: 0.4179 - val_loss: 0.0000e+00 - val_accuracy: 0.4161
Epoch 5/35
297/297 [==============================] - 88s 298ms/step - loss: 0.0000e+00 - accuracy: 0.4179 - val_loss: 0.0000e+00 - val_accuracy: 0.4161

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0003000000142492354.
Epoch 6/35
297/297 [==============================] - 88s 298ms/step - loss: 0.0000e+00 - accuracy: 0.4179 - 

KeyboardInterrupt: 